# machine learning codes for predicting the quality of the wine by using pytorch

In [2]:
import csv
import torch
import numpy as np

In [22]:
wine_path = 'data/winequality-white.csv'
wine_qua_numpy = np.loadtxt(winePath, dtype=np.float32, delimiter=';', skiprows=1)
wine_qua_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

## check that all the data has been read

In [23]:
column_list = next(csv.reader(open(wine_path), delimiter=';'))
wine_qua_numpy.shape, column_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

## convert the NumPy array to a PyTorch tensor

In [24]:
wine_qua = torch.from_numpy(wine_qua_numpy)
wine_qua.shape, wine_qua.type()

(torch.Size([4898, 12]), 'torch.FloatTensor')

## remove the score from the tensor of input data and keep it in a separate tensor

In [25]:
data = wine_qua[:, :-1]
target = wine_qua[:, -1].long()
data.shape, target.shape[0]

(torch.Size([4898, 11]), 4898)

## transform the target tensor in a tensor of labels by one-hot encoding
For each row, take the index of the target label (which coincides with the score in this case), and use it as the column index to set the value 1.0. The result is a tensor encoding categorical information.
The call to unsqueeze adds a singleton dimension, from a 1D tensor of 4898 elements to a 2D tensor of size (4898x1)

In [26]:
target_onehot = torch.zeros(target.shape[0], 10)
# modify the tensor in place
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## use the functions in the PyTorch Tensor API to manipulate your data in tensor form
- obtain means and standard deviations for each column
- look at the data with an eye to finding an easy way to tell good and bad wines apart at a glance.
- get information about wines grouped into good, middling, and bad categories

In [27]:
# dim=0 indicates that the reduction is performed along dimension 0
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [28]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [29]:
# normalize the data by subtracting the mean and dividing by the standard deviation, which helps with the learning process.
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7209e-01, -8.1764e-02,  2.1325e-01,  ..., -1.2468e+00,
         -3.4914e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7991e-02,  ...,  7.3992e-01,
          1.3467e-03, -8.2418e-01],
        [ 1.4756e+00,  1.7448e-02,  5.4378e-01,  ...,  4.7502e-01,
         -4.3677e-01, -3.3662e-01],
        ...,
        [-4.2042e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3131e+00,
         -2.6152e-01, -9.0544e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0048e+00,
         -9.6250e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7502e-01,
         -1.4882e+00,  1.0448e+00]])

lt：less than
le：less than or equal to
eq：equal to
ne：not equal to
ge：greater than or equal to
gt：greater than

In [30]:
# use the torch.le function to determine which rows in target correspond to a score less than or equal to 3
bad_indexes = torch.le(target, 3)
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [31]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [52]:
bad_data = data[torch.le(target, 3)]
mid_data = data[torch.gt(target, 3) & torch.lt(target, 7)]
high_data = data[torch.ge(target, 7)]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
high_mean = torch.mean(high_data, dim=0)

for i, args in enumerate(zip(column_list, bad_mean, mid_mean, high_mean)):
    print ('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [53]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [54]:
actual_indexes = torch.gt(target, 5)
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [56]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches/n_actual, n_matches/n_predicted

(2018, 0.6193984039287906, 0.74000733406674)